In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from util import load_data
from util import train_model
from util import set_seed
from util import TransformerLayer
from util import TransformerEncoder

## Data loader

In [ ]:
spe = "yeast1"

batch_size = 128
lr = 0.001
set_seed(1234)

## local test

# data_dir = "ppi-data"
# kfold_dir = "kfold_20"
# epochs = 10

## google colab test

from google.colab import drive
drive.mount('/content/drive')
data_dir = "drive/MyDrive/ppi-data"
kfold_dir = "kfold_1234"
epochs = 50

device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_h5 = os.path.join(data_dir, spe, "embedding.h5")

## MyPPI

In [ ]:
class Featuring(nn.Module):
    def __init__(self, seq_len, input_dim):
        super().__init__()

        self.ln1 = nn.Linear(seq_len * input_dim, 2048)
        self.bn1 = nn.BatchNorm1d(2048)
        self.ln2 = nn.Linear(2048, 1024)
        self.bn2 = nn.BatchNorm1d(1024)
        self.ln3 = nn.Linear(1024, 512)
        self.bn3 = nn.BatchNorm1d(512)
        self.ln4 = nn.Linear(512, 128)
        self.bn4 = nn.BatchNorm1d(128)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.ln1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.ln2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.dropout(x)
        x = self.ln3(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.dropout(x)
        x = self.ln4(x)
        x = self.relu(x)
        x = self.bn4(x)

        return x


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(128, 8)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(8)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = x.squeeze(-1)
        x = F.sigmoid(x)
        
        return x
    

class InteractionModel(nn.Module):
    def __init__(self, featuring, classifier):
        super().__init__()

        self.featuring = featuring
        self.classifier = classifier

    def forward(self, x1, x2):
        x1 = torch.flatten(x1, start_dim=1)
        x2 = torch.flatten(x2, start_dim=1)
        x1 = self.featuring(x1)
        x2 = self.featuring(x2)
        x = x1 * x2
        x = self.classifier(x)
        return x

for k in range(5):
    print(f"Kfold: ======================== {k+1} ========================")
    train_file = os.path.join(data_dir, spe, kfold_dir, f"train_fold_{k+1}.tsv")
    val_file = os.path.join(data_dir, spe, kfold_dir, f"val_fold_{k+1}.tsv")
    train_loader = load_data(train_file, batch_size, embedding_h5, train=True)
    val_loader = load_data(val_file, batch_size, embedding_h5, train=False)
    
    seq_len = 1500
    input_dim = 13
    featuring = Featuring(seq_len, input_dim)
    classifier = Classifier()
    model = InteractionModel(featuring, classifier).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_model(model, train_loader, val_loader, optimizer, epochs, device)


## PIPR

In [ ]:
class ProteinInteractionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()

        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.gru1 = nn.GRU(hidden_dim, hidden_dim, bidirectional=True,
                           batch_first=True)
        self.conv2 = nn.Conv1d(3 * hidden_dim, hidden_dim, kernel_size=3,
                               padding=1)
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, bidirectional=True,
                           batch_first=True)
        self.conv3 = nn.Conv1d(3 * hidden_dim, hidden_dim, kernel_size=3,
                               padding=1)
        self.gru3 = nn.GRU(hidden_dim, hidden_dim, bidirectional=True,
                           batch_first=True)
        self.conv4 = nn.Conv1d(3 * hidden_dim, hidden_dim, kernel_size=3,
                               padding=1)
        self.gru4 = nn.GRU(hidden_dim, hidden_dim, bidirectional=True,
                           batch_first=True)
        self.conv5 = nn.Conv1d(3 * hidden_dim, hidden_dim, kernel_size=3,
                               padding=1)
        self.gru5 = nn.GRU(hidden_dim, hidden_dim, bidirectional=True,
                           batch_first=True)
        self.conv6 = nn.Conv1d(3 * hidden_dim, hidden_dim, kernel_size=3,
                               padding=1)

        self.pool = nn.MaxPool1d(kernel_size=3, stride=3)
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)
        self.leaky_relu = nn.LeakyReLU(0.3)

        self.fc1 = nn.Linear(hidden_dim, 100)
        self.fc2 = nn.Linear(100, (hidden_dim + 7) // 2)
        self.fc3 = nn.Linear((hidden_dim + 7) // 2, 1)

        self.dropout = nn.Dropout(0.5)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(100)
        self.bn3 = nn.BatchNorm1d((hidden_dim + 7) // 2)

    def process_sequence(self, x):
        # x (b, input_dim, 1500)
        x = self.conv1(x)  # b, hidden_dim, 1500,
        x = self.pool(x)  # b, hidden_dim, 500,
        x = x.permute(0, 2, 1)  # b, 500, hidden_dim
        gru_out, _ = self.gru1(x)  # b, 500, hidden_dim
        x = x.permute(0, 2, 1)  # b, hidden_dim, 500
        gru_out = gru_out.permute(0, 2, 1)  # b, hidden_dim, 500
        x = torch.cat([gru_out, x], dim=1)  # b, 3*hidden_dim, 500

        x = self.conv2(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        gru_out, _ = self.gru2(x)
        x = x.permute(0, 2, 1)
        gru_out = gru_out.permute(0, 2, 1)
        x = torch.cat([gru_out, x], dim=1)  # b, 3*hidden_dim, 166

        x = self.conv3(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        gru_out, _ = self.gru3(x)
        x = x.permute(0, 2, 1)
        gru_out = gru_out.permute(0, 2, 1)
        x = torch.cat([gru_out, x], dim=1)  # b, 3*hidden_dim, 55

        x = self.conv4(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        gru_out, _ = self.gru4(x)
        x = x.permute(0, 2, 1)
        gru_out = gru_out.permute(0, 2, 1)
        x = torch.cat([gru_out, x], dim=1)  # b, 3*hidden_dim, 18

        x = self.conv5(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        gru_out, _ = self.gru5(x)
        x = x.permute(0, 2, 1)
        gru_out = gru_out.permute(0, 2, 1)
        x = torch.cat([gru_out, x], dim=1)  # b, 3*hidden_dim, 6

        x = self.conv6(x)  # b, hidden_dim, 6

        x = self.adaptive_pool(x).squeeze(-1)  # b, hidden_dim
        return x

    def forward(self, x1, x2):
        x1 = x1.permute(0, 2, 1)
        x1 = self.process_sequence(x1)

        x2 = x2.permute(0, 2, 1)
        x2 = self.process_sequence(x2)

        merged = x1 * x2

        x = self.fc1(merged)
        x = self.bn2(x)
        x = self.leaky_relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn3(x)
        x = self.leaky_relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = torch.flatten(x)
        x = F.sigmoid(x)
        return x

for k in range(5):
    print(f"Kfold: ======================== {k+1} ========================")
    train_file = os.path.join(data_dir, spe, kfold_dir, f"train_fold_{k+1}.tsv")
    val_file = os.path.join(data_dir, spe, kfold_dir, f"val_fold_{k+1}.tsv")
    train_loader = load_data(train_file, batch_size, embedding_h5, train=True)
    val_loader = load_data(val_file, batch_size, embedding_h5, train=False)
    
    model = ProteinInteractionModel(input_dim=13, hidden_dim=26).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_model(model, train_loader, val_loader, optimizer, epochs, device)


## DeepFE-PPI

In [ ]:
class MergedDBN(nn.Module):
    def __init__(self, seq_len=1500, input_dim=13):
        super().__init__()

        self.left_branch = nn.Sequential(
            nn.Linear(seq_len * input_dim, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),

            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128)
        )

        self.right_branch = nn.Sequential(
            nn.Linear(seq_len * input_dim, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),

            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128)
        )

        # Combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(256, 8),
            nn.ReLU(),
            nn.BatchNorm1d(8),
            nn.Dropout(0.5),
            nn.Linear(8, 1)
        )

        # L2 regularization will be handled in the optimizer
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x1, x2):
        x1 = torch.flatten(x1, start_dim=1)
        x2 = torch.flatten(x2, start_dim=1)
        x1 = self.left_branch(x1)
        x2 = self.right_branch(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(x)
        x = F.sigmoid(x)
        return x.squeeze()


for k in range(5):
    print(f"Kfold: ======================== {k+1} ========================")
    train_file = os.path.join(data_dir, spe, kfold_dir, f"train_fold_{k+1}.tsv")
    val_file = os.path.join(data_dir, spe, kfold_dir, f"val_fold_{k+1}.tsv")
    train_loader = load_data(train_file, batch_size, embedding_h5, train=True)
    val_loader = load_data(val_file, batch_size, embedding_h5, train=False)
    
    seq_len = 1500
    input_dim = 13
    model = MergedDBN(seq_len, input_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_model(model, train_loader, val_loader, optimizer, epochs, device)


## DeepTrio

In [ ]:
class DeepTrio(nn.Module):
    def __init__(self, em_dim=13, kernel_rate_1=0.16, strides_rate_1=0.15,
                 filter_num_1=150, kernel_rate_2=0.14, strides_rate_2=0.25,
                 filter_num_2=175, con_drop=0.05, fn_drop_1=0.2, fn_drop_2=0.1,
                 node_num=256):
        super(DeepTrio, self).__init__()

        # Create convolution layers for different kernel sizes
        self.conv_layers = nn.ModuleList()
        for n in range(2, 35):
            if n <= 15:
                kernel_size = int(np.ceil(kernel_rate_1 * n ** 2))
                stride = int(np.ceil(strides_rate_1 * (n - 1)))
                conv_layer = nn.Conv1d(
                    in_channels=em_dim,
                    out_channels=filter_num_1,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=0,
                    bias=False
                )
            else:
                kernel_size = int(np.ceil(kernel_rate_2 * n ** 2))
                stride = int(np.ceil(strides_rate_2 * (n - 1)))
                conv_layer = nn.Conv1d(
                    in_channels=em_dim,
                    out_channels=filter_num_2,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=0,
                    bias=False
                )
            self.conv_layers.append(conv_layer)

        self.conv_dropout = nn.Dropout2d(con_drop)
        self.fc_dropout1 = nn.Dropout(fn_drop_1)
        self.fc_dropout2 = nn.Dropout(fn_drop_2)

        # Calculate the total number of features after concatenation
        total_features = 0
        for n in range(2, 35):
            if n <= 15:
                total_features += filter_num_1
            else:
                total_features += filter_num_2

        self.fc1 = nn.Linear(total_features, node_num)
        self.fc2 = nn.Linear(node_num, 1)

    def forward(self, x1, x2):

        # Permute for Conv1d: (batch, channels, seq_len)
        x1 = x1.permute(0, 2, 1)
        x2 = x2.permute(0, 2, 1)

        tensor = []

        for i, conv_layer in enumerate(self.conv_layers):
            # Apply convolution
            conv_out_1 = F.relu(conv_layer(x1))
            conv_out_2 = F.relu(conv_layer(x2))

            # Apply dropout
            conv_out_1 = self.conv_dropout(conv_out_1.unsqueeze(-1)).squeeze(-1)
            conv_out_2 = self.conv_dropout(conv_out_2.unsqueeze(-1)).squeeze(-1)

            # Apply max pooling
            pool_out_1 = F.max_pool1d(conv_out_1, conv_out_1.size(-1))
            pool_out_2 = F.max_pool1d(conv_out_2, conv_out_2.size(-1))

            # Flatten
            flat_out_1 = pool_out_1.view(pool_out_1.size(0), -1)
            flat_out_2 = pool_out_2.view(pool_out_2.size(0), -1)

            pool_out = flat_out_1 + flat_out_2

            tensor.append(pool_out)

        # Concatenate all features
        concatenated = torch.cat(tensor, dim=1)

        # Fully connected layers
        x = self.fc_dropout1(concatenated)
        x = self.fc1(x)
        x = self.fc_dropout2(x)
        x = F.relu(x)
        x = self.fc2(x)

        return F.sigmoid(x).squeeze()

epochs = 30

for k in range(5):
    print(f"Kfold: ======================== {k+1} ========================")
    train_file = os.path.join(data_dir, spe, kfold_dir, f"train_fold_{k+1}.tsv")
    val_file = os.path.join(data_dir, spe, kfold_dir, f"val_fold_{k+1}.tsv")
    train_loader = load_data(train_file, batch_size, embedding_h5, train=True)
    val_loader = load_data(val_file, batch_size, embedding_h5, train=False)

    model = DeepTrio().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_model(model, train_loader, val_loader, optimizer, epochs, device)

## TF-PPI

In [ ]:
class Featuring(nn.Module):
    def __init__(self, input_dim, feature_dim):
        super().__init__()

        self.feature_dim = feature_dim

        self.conv1 = nn.Conv1d(input_dim, feature_dim, kernel_size=3,
                               padding=1)
        layer1 = TransformerLayer(n_heads=3, d_model=feature_dim,
                                  ff_units=10, dropout=0.2)
        self.encoder1 = TransformerEncoder(layer1, n_layers=2)

        self.conv2 = nn.Conv1d(2 * feature_dim, feature_dim, kernel_size=3,
                               padding=1)
        layer2 = TransformerLayer(n_heads=3, d_model=feature_dim,
                                  ff_units=10, dropout=0.2)
        self.encoder2 = TransformerEncoder(layer2, n_layers=2)

        self.conv3 = nn.Conv1d(2 * feature_dim, feature_dim, kernel_size=3,
                               padding=1)
        layer3 = TransformerLayer(n_heads=3, d_model=feature_dim,
                                  ff_units=10, dropout=0.2)
        self.encoder3 = TransformerEncoder(layer3, n_layers=2)

        self.conv4 = nn.Conv1d(2 * feature_dim, feature_dim, kernel_size=3,
                               padding=1)

        self.pool = nn.MaxPool1d(kernel_size=3, stride=3)
        # self.adaptive_pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # b, input_dim, 1500

        # First layer
        x = self.conv1(x)  # b, feature_dim, 1500,
        x = self.pool(x)  # b, feature_dim, 500
        x = x.permute(0, 2, 1)  # n, 500, feature_dim
        e = self.encoder1(x)  # b, 500, feature_dim
        x = x.permute(0, 2, 1)  # b, feature_dim, 500
        e = e.permute(0, 2, 1)  # b, feature_dim, 500
        x = torch.cat([e, x], dim=1)  # b, 2*feature_dim, 500

        # Second layer
        x = self.conv2(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        e = self.encoder2(x)
        x = x.permute(0, 2, 1)
        e = e.permute(0, 2, 1)
        x = torch.cat([e, x], dim=1)  # b, 2*feature_dim, 166

        # Third layer
        x = self.conv3(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        e = self.encoder3(x)
        x = x.permute(0, 2, 1)
        e = e.permute(0, 2, 1)
        x = torch.cat([e, x], dim=1)  # b, 2*feature_dim, 55

        x = self.conv4(x)  # b, feature_dim, 55
        x = x.permute(0, 2, 1)  # b, 55, feature_dim

        return x


class InteractionModel(nn.Module):
    def __init__(self, featuring):
        super().__init__()

        self.featuring = featuring
        self.feature_dim = featuring.feature_dim

        layer = TransformerLayer(n_heads=3,
                                 d_model=self.feature_dim,
                                 ff_units=2*self.feature_dim,
                                 dropout=0.5)
        self.encoder = TransformerEncoder(layer, n_layers=2)

        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),

            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),

            nn.Linear(128, 8),
            nn.ReLU(),
            nn.BatchNorm1d(8),
            nn.Dropout(0.5),

            nn.Linear(8, 1)
        )

    def forward(self, x1, x2):
        x1 = self.featuring(x1)  # b, 55, feature_dim
        x2 = self.featuring(x2)  # b, 55, feature_dim
        x = torch.cat((x1, x2), dim=1)  # b, 110, feature_dim
        x = self.encoder(x) # b, 110, feature_dim
        x = torch.mean(x, dim=1)  # b, feature_dim
        x = self.classifier(x)
        x = F.sigmoid(x)
        return x.squeeze()


for k in range(5):
    print(f"Kfold: ======================== {k+1} ========================")
    train_file = os.path.join(
        data_dir, spe, kfold_dir, f"train_fold_{k+1}.tsv")
    val_file = os.path.join(data_dir, spe, kfold_dir, f"val_fold_{k+1}.tsv")
    train_loader = load_data(train_file, batch_size, embedding_h5, train=True)
    val_loader = load_data(val_file, batch_size, embedding_h5, train=False)

    input_dim = 13
    feature_dim = 24
    featuring = Featuring(input_dim, feature_dim)
    model = InteractionModel(featuring).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_model(model, train_loader, val_loader, optimizer, epochs, device)